In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-09-14 08:01:54.956955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-14 08:01:54.957072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-14 08:01:55.150855: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")
print("wandb")
wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune_Llama31_8B', 
    job_type="training", 
    anonymous="allow"
)
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
dataset_name = "/kaggle/input/dataset-agriculture-pc-k18/dataset_agriculture_pc_18k.xlsx"
new_model = "llama-31-8b-ft-agri"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
wandb


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elbrahimimeriem (elbrahimimeriem-university hassan 2). Use `wandb login --relogin` to force relogin


In [5]:
torch_dtype = torch.float16
attn_implementation = "eager"
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [1]:
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)
import pandas as pd
from datasets import Dataset

dataset = pd.read_excel(dataset_name)
dataset


,Prompt,Context,Prompt_context
0,Prunier: Taille de formation,Il faut intervenir sur les pruniers les 7 prem...,Prunier: Taille de formation \n Il faut interv...
1,Quels outils utiliser pour la taille d'un prun...,"Pour effectuer la taille d'un prunier, il faut...",Quels outils utiliser pour la taille d'un prun...
2,Comment procéder à la taille d'un prunier ?,"Pour tailler un prunier efficacement, il suffi...",Comment procéder à la taille d'un prunier ? \n...
3,Semis : comment bien choisir son noyau de pêche ?,"Avant de planter un noyau, choisir une pêche d...",Semis : comment bien choisir son noyau de pêch...
4,Pourquoi installent-on des pêchers dans les vi...,Les pêches mûrissent au moment des vendanges. ...,Pourquoi installent-on des pêchers dans les vi...
...,...,...,...
18204,Pourquoi choisir une zone large et ensoleillée...,Il est important de choisir un endroit large a...,Pourquoi choisir une zone large et ensoleillée...
18205,Quel est le sol approprié pour la culture des ...,Si votre sol a été utilisé pour cultiver des f...,Quel est le sol approprié pour la culture des ...
18206,Comment protéger les plants de citrouilles des...,Il est important de choisir un endroit pour pl...,Comment protéger les plants de citrouilles des...
18207,Quel est le meilleur moment pour planter des c...,Les citrouilles sont très sensibles aux gelées...,Quel est le meilleur moment pour planter des c...


In [7]:
from sklearn.model_selection import train_test_split

# Définir la fonction format_chat_template avec gestion des erreurs
def format_chat_template(row):
    try:
        # Ajouter un rôle système dédié à l'agriculture au Maroc
        row_json = [
            {"role": "system", "content": "Tu es un assistant virtuel dédié à l'agriculture au Maroc. Tu aides les agriculteurs en leur fournissant  des réponses détaillées et adaptées aux conditions locales."},
            {"role": "user", "content": row["Prompt"]},
            {"role": "assistant", "content": row["Context"]}
        ]
         # Appliquer le modèle de chat en utilisant le tokenizer
        row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    except Exception as e:
        print(f"Error processing row: {e}")
        print(f"Row data: {row}")
    return row

# Appliquer la fonction au DataFrame nettoyé
dataset = dataset.apply(format_chat_template, axis=1)


# Diviser le dataset en train et test
train_df, test_df = train_test_split(dataset, test_size=0.009, random_state=42)

# Convertir le DataFrame pandas en Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [8]:
# Vérifier le résultat
print(dataset['text'][233])


<|im_start|>system
Tu es un assistant virtuel dédié à l'agriculture au Maroc. Tu aides les agriculteurs en leur fournissant  des réponses détaillées et adaptées aux conditions locales.<|im_end|>
<|im_start|>user
La stratification des noyaux d'olive : pas-à-pas<|im_end|>
<|im_start|>assistant
Choisir des olives très mûres, noires, molles et prêtes à tomber. Gratter le noyau avec du papier de verre ou une lime à ongles. Tremper les graines dans une eau tiède pendant 48h. Mélanger les noyaux dans un pot rempli de sable légèrement humide. Enterrer le pot au pied d'un mur exposé au nord recouvert d'un grillage et des feuilles mortes.<|im_end|>



In [9]:

training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb",
    save_steps=0
) 


# Define SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/18045 [00:00<?, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
1805,1.689000,1.247787
3610,1.501200,1.167980
5415,1.711300,1.112895
7220,1.005500,1.065341


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=9022, training_loss=1.1511795892072765, metrics={'train_runtime': 19762.5667, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.457, 'total_flos': 2.086332032410583e+17, 'train_loss': 1.1511795892072765, 'epoch': 0.999944582986977})

In [22]:
from datetime import datetime

# Obtenir l'heure actuelle
heure_actuelle = datetime.now().time()

# Afficher l'heure actuelle
print("L'heure actuelle est :", heure_actuelle)

messages = [
    {
        "role": "user",
        "content": "quelles régions sont appropriées pour la culture des fèves au maroc"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(
    **inputs,
    max_length=550,
    
    temperature=0.7,
    top_p=0.9, 
    no_repeat_ngram_size=3,  # Avoid repeating n-grams
    num_beams=5,  # Use beam search for completeness
    early_stopping=True # Ajouter une pénalité de répétition
)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)
a=text.split("assistant")[1]
print(a.split("Source :")[0])
heure_actuelle = datetime.now().time()
print("L'heure actuelle est :", heure_actuelle)

L'heure actuelle est : 13:54:38.204280

Les régions les plus adaptées à la culture de la fève au Maroc sont le Gharb, le Loukkos, le Souss-Massa et le Haouz. Ces régions bénéficient d'un climat doux et d'un sol riche en nutriments, ce qui favorise la croissance et le rendement des cultures. La fève est également cultivée dans d'autres régions du Maroc, mais ces régions présentent des conditions climatiques et édaphiques plus défavorables. Il est important de choisir les régions appropriées en fonction des caractéristiques du sol et des conditions météorologiques pour assurer une croissance optimale et un rendement élevé.  
L'heure actuelle est : 13:57:03.600297


In [10]:
model.save_pretrained("llama3.1_agri")
tokenizer.save_pretrained("llama3.1_agri")
#trainer.model.config.push_to_hub("laila1234/fine-tuned-llama3-300l-without config")
tokenizer.push_to_hub("MeelUnv/llama3.1_agri")
model.push_to_hub("MeelUnv/llama3.1_agri")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MeelUnv/llama3.1_agri/commit/b560529ab4dfc7a1e792152bb914489e4dac2e09', commit_message='Upload model', commit_description='', oid='b560529ab4dfc7a1e792152bb914489e4dac2e09', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
config = trainer.model.config
config.push_to_hub("MeelUnv/llama3.1_agri")

CommitInfo(commit_url='https://huggingface.co/MeelUnv/llama3.1_agri/commit/6aaf5adf5301311dd92526819189cede994bbe79', commit_message='Upload config', commit_description='', oid='6aaf5adf5301311dd92526819189cede994bbe79', pr_url=None, pr_revision=None, pr_num=None)